In [ ]:
import numpy as np

## Topology-aware neuroanatomical analysis

Neurons are, in general, tree-like structures. While there are fine scale anatomical details like spines and boutons, their anatomy can be largely described as a collection of branched linear processes and a roughly spherical cell body (or soma). This representation of a neuron is called a "skeleton". Because of this, many of the topological aspects of trees in general are useful for neurons as well, for example defining "branch points" and "end points" or "tips". Moreover, the notion of a "root" — in our case the soma — lets us define a base point from which we can define notions of "downstream" and "upstream". Finally, because a tree has no loops, there is only one possible path between any two points on a skeleton and thus many computations are much faster than on a much larger mesh.

Here, we explore the meshparty implementation of such topologically aware data structures. First, we will look at basic Skeletons, which handle the operations like those mentioned above. Second, we will look at "Meshworks", objects that combine meshes, skeletons, and annotations to try to simply how these three useful types of data work together.

## 1. Skeletons

### Defining a Skeleton
 
At its simplest, MeshParty can build a skeleton from vertices --- points in 3d space --- and edges that link them. Vertices are defined as an Nx3 numpy array of xyz coordinates, while edges are an Mx2 numpy array of vertex indices to link. Let's start with a simple Y-shaped example.

In [ ]:
from meshparty import skeleton

verts = np.array([[10, 0, 50],
                  [10, 10, 50],
                  [10, 20, 50],
                  [0, 20, 50],
                  [0, 20, 40],
                  [20, 20, 50],
                  [20, 20, 60]])

edges = np.array([[0, 1],
                  [1, 2],
                  [2, 3],
                  [3, 4],
                  [2, 5],
                  [5, 6]], dtype=int)

simple_sk = skeleton.Skeleton(verts, edges)

A skeleton's vertices and edges are stored in the object under the properties `vertices` and `edges`, respectively.

In [ ]:
simple_sk.vertices

In [ ]:
simple_sk.edges

All skeletons have a _root_ index that defines the "base" of the skeleton and adds directionality to the edges. We can specify a root on skeleton creation or reroot to a new index at any time.

In [ ]:
print('Initial Root: ', simple_sk.root)
first_root = simple_sk.root
simple_sk.reroot(6)
print('New Root: ', simple_sk.root)
simple_sk.reroot(first_root)

Each vertex other than the root has a single 'parent', the next vertex closer to the root. We can get the parent nodes of one or more vertex indices with `parent_nodes`. Note that the parent node of the root is given a value of -1.

In [ ]:
print('Parent nodes: ', simple_sk.parent_nodes(np.array([0,1,2,3])))

Vertices also have "children". Nodes with no children are "end" nodes, the tips of the the skeleton, while nodes with more than one child are "branch nodes". We can get child nodes for specific vertices with `child_nodes`. Note that child_nodes returns a list of arrays, since each vertex can have between 0 and an arbitrary number of childen.

In [ ]:
print('Child nodes nodes: ',simple_sk.child_nodes(np.array([0,2,6])))

Because points like ends and branches are very important, and we often want to get to list them. An array of branch point indices is at `branch_points` and end points are at `end points`. Note that `end_points` only returns ends that have no children, and thus would exclude a root that is otherwise tip-like. In the case that you want all tip-like ends, `end_points_undirected` will return all vertices with only a single parent or child.

In [ ]:
print('Branch point indices: ', simple_sk.branch_points)
print('End point indices: ', simple_sk.end_points)

Like meshes, we often want to consider the graph representation of the skeleton. The property `csgraph` returns a [Scipy sparse matrix](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html) of the directed adjacency matrix of the skeleton where the i,jth entry is the distance from vertex i to vertex j if and only if vertex j is the parent of vertex i.

In [ ]:
G = simple_sk.csgraph
G.toarray() # just to see the matrix.

Undirected, unweighted, and both undirected and unweighted versions are available as well: `csgraph_undirected`, `csgraph_binary`, and `csgraph_binary_undirected`.

### Getting skeletons from meshes or files

The two principle ways to get a neuronal skeleton are from skeletonizing a mesh or from loading a precomputed skeleton. MeshParty can compute a skeleton from a mesh with the `skeletonize` submodule. The algorithm and parameters involved in skeletonization are discussed at length in [their own section](https://meshparty.readthedocs.io/en/latest/guide/skeletons.html#skeletonization).

Since computing a skeleton from a large mesh can take up to a few minutes, it's useful to save pre-computed skeletons. Similar to meshes, we save skeletons in an hdf5 format that can be loaded with the `skeleton_io.read_skeleton_h5` file. There is an example file in `data/`. The equivalent function for writing skeleton `sk` is `sk.write_to_h5`

In [ ]:
from meshparty import skeleton_io

sk = skeleton_io.read_skeleton_h5('data/648518346349515986_skeleton.h5')

### Visualizing skeletons

For starters, we want to be able to look at skeletons in 3d. The `trimesh_vtk.skeleton_actor` function produces a VTK actor, like the `trimesh_vtk.mesh_actor` function for meshes. In order to show a few more points of interest, let's highlight the root node (soma), branch nodes, and end nodes. We can use the `trimesh_vtk.point_cloud_actor` function to generate N spheres given an Nx3 list of locations. Note that units are in nanometers.

In [ ]:
from meshparty import trimesh_vtk

sk_actor = trimesh_vtk.skeleton_actor(sk, line_width=3, color=(0,0,0))
root_actor = trimesh_vtk.point_cloud_actor(sk.vertices[sk.root].reshape(1,3), size=10000, color=(0,0,0), opacity=0.5)  # point_cloud_actor needs an Nx3 input array
branch_actor = trimesh_vtk.point_cloud_actor(sk.vertices[sk.branch_points], size=1000, color=(1,0,0), opacity=1)
end_actor = trimesh_vtk.point_cloud_actor(sk.vertices[sk.end_points], size=1000, color=(0,0,1), opacity=1)

trimesh_vtk.render_actors([sk_actor, root_actor, branch_actor, end_actor])

### Upstream, downstream, and on-path vertices

For a given vertex, we can quickly get all vertices downstream (away from root) with `downstream_nodes`. Let's use this function to pick one of the immediate child nodes of the soma (i.e. one of the primary processes) and highlight it.

In [ ]:
some_index = sk.child_nodes(sk.root)[2]

downstream_indices = sk.downstream_nodes(some_index)

# Visualize the branch by adding red dots to all downstream indices
sk_actor = trimesh_vtk.skeleton_actor(sk, line_width=3, color=(0,0,0))
ds_highlight = trimesh_vtk.point_cloud_actor(sk.vertices[downstream_indices], size=500, color=(1,0.2,0.2), opacity=1)
trimesh_vtk.render_actors([sk_actor, ds_highlight])

Similarly, we can start from any node and find the path to root. Note that this includes just the direct path from that node to the root and does not go down other branch points.

In [ ]:
some_index = sk.end_points[0]

path_indices = sk.path_to_root(some_index)

# Visualize the branch by adding blue dots to all indices on the path
sk_actor = trimesh_vtk.skeleton_actor(sk, line_width=3, color=(0,0,0))
ds_highlight = trimesh_vtk.point_cloud_actor(sk.vertices[path_indices], size=500, color=(0.2,0.2,1), opacity=1)
trimesh_vtk.render_actors([sk_actor, ds_highlight])

In fact, there is nothing unique about root. It's fast to find the direct path between any two indices. on the skeleton.

In [ ]:
index_a = sk.end_points[0]
index_b = sk.end_points[20]
path_indices = sk.path_between(index_a, index_b)

# Visualize the branch by adding green dots to all indices along the path
sk_actor = trimesh_vtk.skeleton_actor(sk, line_width=3, color=(0,0,0))
ds_highlight = trimesh_vtk.point_cloud_actor(sk.vertices[path_indices], size=500, color=(0.2,1,0.2), opacity=1)
start_end_highlight = trimesh_vtk.point_cloud_actor(sk.vertices[[index_a, index_b]], size=5000, color=(0.2,1,0.2), opacity=0.2)
trimesh_vtk.render_actors([sk_actor, ds_highlight, start_end_highlight])

### Distances

Skeletons let one quickly compute distances. In general, one can compute the total path length spanned by a collection of indices with the `path_length` function. For example, to get the path length of the path found above:

In [ ]:
index_a = sk.end_points[0]
index_b = sk.end_points[20]
path_indices = sk.path_between(index_a, index_b)

sk.path_length(path_indices)

The distance between any node and the soma is a property we want frequently enough that it's pre-computed as an array at `distance_to-root`. For example, if we want the distance to root for every end point:

In [ ]:
sk.distance_to_root[sk.end_points]

### Segments

Each span of a skeleton without branch points (i.e. branch point to end point or branch point to branch point) is called a 'segment'. The whole neuron can be split into non-overlapping segments, and this can be handy for certain computations. A list of arrays for the indices in each segment is at `segments'.

In [ ]:
some_seg = sk.segments[50]

# Visualize the branch by adding green dots to all indices in the segment
sk_actor = trimesh_vtk.skeleton_actor(sk, line_width=3, color=(0,0,0))
highlight = trimesh_vtk.point_cloud_actor(sk.vertices[some_seg], size=500, color=(0.2,1,0.2), opacity=1)
trimesh_vtk.render_actors([sk_actor, highlight])

The list of segments is not organized in any particular way, so if you want to look up which segment a given index belongs to, use the `segment_map` property. This is an array that provides the segment index a given vertex index belongs to.

In [ ]:
index_a = 10080

seg_ind = sk.segment_map[index_a]
some_seg = sk.segments[seg_ind]

# Visualize the segment by highlighting all points on the same segment as a red dot.
sk_actor = trimesh_vtk.skeleton_actor(sk, line_width=3, color=(0,0,0))
base_point = trimesh_vtk.point_cloud_actor(sk.vertices[index_a].reshape(1,3), size=3000, color=(1,0.2,0.2), opacity=1)
highlight = trimesh_vtk.point_cloud_actor(sk.vertices[some_seg], size=500, color=(0.2,1,0.2), opacity=1)
trimesh_vtk.render_actors([sk_actor, highlight, base_point])

### Cover Paths

Let's say you want to traverse a complete skeleton along a collection of non-branching paths that all move toward root. Since each end point must be represented, the minimal number of such paths is the same as the number of end points. It turns out that a useful way to build the exact set of paths is to use the following procedure:
    
    1) Start a new path at the farthest end point index that hasn't been visited yet.
    
    2) Add indices as you move toward root until you either hit root or a point that's been visited already.
    
    3) Finish when all indices have been visited.

The convenience of this approach is that if you traverse this path list in reverse order (starting from the first point of last path created), you will never traverse a branch point before visiting all vertices downstream of it. This allows a number of useful algorithms to run in a single iteration across the nodes of the skeleton. A list of arrays of such cover paths is at `cover_paths`.

In [ ]:
some_cover_path = sk.cover_paths[12]

sk_actor = trimesh_vtk.skeleton_actor(sk, line_width=3, color=(0,0,0))
highlight = trimesh_vtk.point_cloud_actor(sk.vertices[some_cover_path], size=500, color=(0.2,1,0.2), opacity=1)
trimesh_vtk.render_actors([sk_actor, highlight])